In [95]:
import pandas as pd
import numpy as np
import pathlib as pl
# library used for initial visualization
# library used for initial visualization
import sklearn
import pathlib as pl

In [96]:
path = 'C:/Users/Rolando/Downloads/financialphrases/all-data.csv'

df = pd.read_csv(path, encoding = "ISO-8859-1", names = ['sentiment', 'sentence'])

In [97]:
df

,sentiment,sentence
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [98]:
df['sentence']

0       According to Gran , the company has no plans t...
1       Technopolis plans to develop in stages an area...
2       The international electronic industry company ...
3       With the new production plant the company woul...
4       According to the company 's updated strategy f...
                              ...                        
4841    LONDON MarketWatch -- Share prices ended lower...
4842    Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843    Operating profit fell to EUR 35.4 mn from EUR ...
4844    Net sales of the Paper segment decreased to EU...
4845    Sales in Finland decreased by 10.5 % in Januar...
Name: sentence, Length: 4846, dtype: object

In [99]:
# path = 'C:/Users/Rolando/Downloads/newsTitles/SEntFiN-v1.1.csv'

# df = pd.read_csv(path, encoding = "ISO-8859-1")

In [100]:
# df

In [101]:
# path = 'C:/Users/Rolando/Downloads/newsTitles2/data.csv'

# df = pd.read_csv(path, encoding = "ISO-8859-1")

In [102]:
# df

In [103]:
# !pip install -U spacy

In [104]:
text = df.sample(1)['sentence'].iloc[0]

SPACY

In [105]:
import spacy

nlp = spacy.load("en_core_web_md")
doc = nlp(text)
print(doc.ents)

(12-month, EUR0, 2004)


RAKE NLTK

In [106]:
from rake_nltk import Rake
rake_nltk_var = Rake()

rake_nltk_var.extract_keywords_from_text(text)
keyword_extracted = rake_nltk_var.get_ranked_phrases()
print(keyword_extracted)

['month period decreased', 'turnover decreased', 'operating result', 'profit', 'eur5', 'eur0', 'compared', '6', '4', '2004', '12']


TFIDF

In [107]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as sw

def train_keyword(data): #SE PUEDE EXTENDER EL DICCIONARIO
#     # use stopword to filter words    
#     stop = sw.stopwords_key()
#     stop = stop.splitlines()
#     stop = str(stop)
#     stop = stop.translate(str.maketrans('', '', string.punctuation))
    stopwords_list = list(sw.words('english'))
#     stopwords_list.extend(stop.split())
    
    # vectorizing the words use tf-idf
    vectorizer = TfidfVectorizer(analyzer='word', stop_words=set(stopwords_list), use_idf=True)
    vector = vectorizer.fit_transform(data)
    
    # store to dataframe    
    df = pd.DataFrame(vector[0].T.todense(), index=vectorizer.get_feature_names(), columns=["vector"])
    df = df.sort_values('vector', ascending=False)
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'keywords'})
    df = df[df['vector'] != 0]

    return df

In [108]:
raw_data = [text]
keywords = train_keyword(raw_data)
# print top 10 keywords
print(keywords.head(10))

    keywords    vector
0  decreased  0.516398
1         12  0.258199
2       2004  0.258199
3   compared  0.258199
4       eur0  0.258199
5       eur5  0.258199
6      month  0.258199
7  operating  0.258199
8     period  0.258199
9     profit  0.258199


KEYBERT

In [109]:
# # Install the libraries
# !pip install keybert
# !pip install sentence-transformers

In [110]:
# Import them
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

# To show the execution times
# !pip install ipython-autotime
# %load_ext autotime

In [111]:
sent_trans = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
keyBERT_model = KeyBERT(model = sent_trans)

In [112]:
def extract_terms(document, n_gram_range = (3,3), 
                  top_N=5, model=keyBERT_model, diversity_threshold = 0.7):

  keywords = model.extract_keywords(document, stop_words='english', 
                                    keyphrase_ngram_range=(1, 3),
                                    use_mmr=True, 
                                    diversity = diversity_threshold,
                                    top_n = top_N)
  
  return sorted(keywords, key=lambda tup:(-tup[1], tup[0]))

In [113]:
print(f'Text: {text}.\nLength: {len(text.split())}')
best_terms = extract_terms(text)

for terms_and_score in best_terms:
  print(terms_and_score)

Text: Operating result for the 12-month period decreased from the profit of EUR0 .4 m while turnover decreased from EUR5 .6 m , as compared to 2004 ..
Length: 27
('turnover decreased eur5', 0.6664)
('operating result', 0.4047)
('12 month period', 0.1871)
('2004', 0.1471)
('compared', 0.0327)


In [114]:
# !pip install git+https://github.com/LIAAD/yake

In [115]:
import yake

In [116]:
kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(text)

for kw in keywords:
	print(kw)

('Operating result', 0.019943439662486344)
('period decreased', 0.021586758591933864)
('turnover decreased', 0.03586555834533851)
('decreased', 0.09671664173256625)
('Operating', 0.10923981997294611)
('period', 0.10923981997294611)
('result', 0.17899586521379354)
('profit', 0.17899586521379354)
('turnover', 0.17899586521379354)
('compared', 0.17899586521379354)


In [117]:
language = "en"
max_ngram_size = 3
deduplication_threshold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)

for kw in keywords:
    print(kw)

('Operating result', 0.019943439662486344)
('period decreased', 0.021586758591933864)
('turnover decreased', 0.03586555834533851)
('decreased', 0.09671664173256625)
('Operating', 0.10923981997294611)
('period', 0.10923981997294611)
('result', 0.17899586521379354)
('profit', 0.17899586521379354)
('turnover', 0.17899586521379354)
('compared', 0.17899586521379354)
